# Intersect catchment with ESA soil moisture project
# calculates correlation between ESA_SM soil moisture and simulated soil moisture in each HRU in the model setup

In [159]:
# modules
import os
import sys
from pathlib import Path
from shutil import copyfile
from datetime import datetime
import geopandas as gpd
import rasterstats
import pandas as pd
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import regionmask
import dask


#### Control file handling

In [160]:
# Easy access to control file folder
controlFolder = Path('../../0_control_files')

In [161]:
# Store the name of the 'active' file in a variable
controlFile = 'control_Yukon_Merit.txt'

In [162]:
# Function to extract a given setting from the control file
def read_from_control( file, setting ):
    
    # Open 'control_active.txt' and ...
    with open(file) as contents:
        for line in contents:
            
            # ... find the line with the requested setting
            if setting in line and not line.startswith('#'):
                break
    
    # Extract the setting's value
    substring = line.split('|',1)[1]      # Remove the setting's name (split into 2 based on '|', keep only 2nd part)
    substring = substring.split('#',1)[0] # Remove comments, does nothing if no '#' is found
    substring = substring.strip()         # Remove leading and trailing whitespace, tabs, newlines
       
    # Return this value    
    return substring

In [163]:
# Function to specify a default path
def make_default_path(suffix):
    
    # Get the root path
    rootPath = Path( read_from_control(controlFolder/controlFile,'root_path') )
    
    # Get the domain folder
    domainName = read_from_control(controlFolder/controlFile,'domain_name')
    domainFolder = 'domain_' + domainName
    
    # Specify the forcing path
    defaultPath = rootPath / domainFolder / suffix
    
    return defaultPath

#### Find location of shapefile and DEM

In [164]:
# Catchment shapefile path & name
catchment_path = read_from_control(controlFolder/controlFile,'catchment_shp_path')
catchment_name = read_from_control(controlFolder/controlFile,'catchment_shp_name')

In [165]:
# Specify default path if needed
if catchment_path == 'default':
    catchment_path = make_default_path('shapefiles/catchment') # outputs a Path()
else:
    catchment_path = Path(catchment_path) # make sure a user-specified path is a Path()

In [166]:
# DEM path & name
mod10a1_path = read_from_control(controlFolder/controlFile,'observation_snow_mod10a1_path')
mod10a1_name = read_from_control(controlFolder/controlFile,'observation_snow_mod10a1_name')

In [167]:
# Specify default path if needed
if mod10a1_path == 'default':
    mod10a1_path = make_default_path('observations/MOD10A1/6_tif_multiband') # outputs a Path()
else:
    mod10a1_path = Path(mod10a1_path) # make sure a user-specified path is a Path()

mod10a1_path

PosixPath('/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/observations/MOD10A1/6_tif_multiband')

#### Find where the intersection needs to go

In [168]:
# Intersected shapefile path and name
intersect_path = read_from_control(controlFolder/controlFile,'intersect_mod10a1_path')
intersect_name = read_from_control(controlFolder/controlFile,'intersect_mod10a1_name')
print(intersect_name)
print(intersect_path)

catchment_with_mod10a1.shp
default


In [169]:
# Specify default path if needed
if intersect_path == 'default':
    intersect_path = make_default_path('shapefiles/catchment_intersection/with_mod10a1') # outputs a Path()
else:
    intersect_path = Path(intersect_path) # make sure a user-specified path is a Path()

In [170]:
# Make the folder if it doesn't exist
intersect_path.mkdir(parents=True, exist_ok=True)

#### Copy the source catchment shapefile into the destination location

In [171]:
# Find the name without extension
catchment_base = catchment_name.replace('.shp','')

In [172]:
# Loop over directory contents and copy files that match the filename of the shape
for file in os.listdir(catchment_path):
    if catchment_base in file: # copy only the relevant files in case there are more than 1 .shp files
        
        # make the output file name
        _,ext = os.path.splitext(file)                    # extension of current file
        basefile,_ = os.path.splitext(intersect_name)     # name of the intersection file w/o extension
        newfile = basefile + ext                          # new name + old extension
        
        # copy
        copyfile(catchment_path/file, intersect_path/newfile);

## Get the SUMMA output file

In [173]:
simulation_path = read_from_control(controlFolder/controlFile,'experiment_output_summa')
simulation_name = read_from_control(controlFolder/controlFile,'experiment_id')

In [174]:
# Specify default path if needed
if simulation_path == 'default':
    simulation_path = make_default_path('simulations/' + simulation_name + '/SUMMA/' + simulation_name + '_day.nc')
    simulation_path = Path(simulation_path) # make sure a user-specified path is a Path()

simulation_path

PosixPath('/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/simulations/run_Yukon_Merit_1/SUMMA/run_Yukon_Merit_1_day.nc')

## correlation analysis analysis

In [175]:

ds = xr.open_mfdataset(simulation_path)

In [176]:
ds

<xarray.Dataset> Size: 16GB
Dimensions:               (time: 15705, hru: 18225, gru: 18225)
Coordinates:
  * time                  (time) datetime64[ns] 126kB 1980-01-02 ... 2022-12-31
  * hru                   (hru) int64 146kB 81001460 81001703 ... 81036235
  * gru                   (gru) int64 146kB 81001460 81001703 ... 81036235
Data variables:
    scalarSWE             (time, hru) float64 2GB dask.array<chunksize=(1, 18225), meta=np.ndarray>
    scalarAquiferStorage  (time, hru) float64 2GB dask.array<chunksize=(1, 18225), meta=np.ndarray>
    scalarTotalSoilWat    (time, hru) float64 2GB dask.array<chunksize=(1, 18225), meta=np.ndarray>
    scalarInfiltration    (time, hru) float64 2GB dask.array<chunksize=(1, 18225), meta=np.ndarray>
    scalarTotalET         (time, hru) float64 2GB dask.array<chunksize=(1, 18225), meta=np.ndarray>
    scalarTotalRunoff     (time, hru) float64 2GB dask.array<chunksize=(1, 18225), meta=np.ndarray>
    scalarNetRadiation    (time, hru) float64 2GB dask.array<chunksize=(1, 18225), meta=np.ndarray>
    hruId                 (hru) int64 146kB dask.array<chunksize=(18225,), meta=np.ndarray>
    gruId                 (gru) int64 146kB dask.array<chunksize=(18225,), meta=np.ndarray>

#### Spatial analysis

In [177]:
# Convert Path() to string for QGIS
catchment_file = str(intersect_path/intersect_name) # needs to be the coped file because output is automatically added to this
mod10a1_file = str(mod10a1_path/mod10a1_name)

In [178]:
layer_polygon = catchment_file
layer_SM = '/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/observations/RS_Soil_moisture/ESA_SM/dap.ceda.ac.uk/neodc/esacci/soil_moisture/data/daily_files/ACTIVE/v08.1/2005/ESACCI-SOILMOISTURE-L3S-SSMS-ACTIVE-20051201000000-fv08.1.nc'


print(layer_SM)
print(layer_polygon)

/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/observations/RS_Soil_moisture/ESA_SM/dap.ceda.ac.uk/neodc/esacci/soil_moisture/data/daily_files/ACTIVE/v08.1/2005/ESACCI-SOILMOISTURE-L3S-SSMS-ACTIVE-20051201000000-fv08.1.nc
/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/shapefiles/catchment_intersection/with_mod10a1/catchment_with_mod10a1.shp


In [179]:
bbox = read_from_control(controlFolder/controlFile,'forcing_raw_space').split('/')
lat_max = bbox[0]
lon_min = bbox[1]
lat_min = bbox[2]
lon_max = bbox[3]

In [180]:
#Loop over all the files inthe directory
dailyPath = '/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/observations/RS_Soil_moisture/ESA_SM/3_daily_correlation/'
files = []
for file in os.listdir(dailyPath):
    # check only text files
    if file.endswith('.nc'):
        #print(file)
        files.append(dailyPath + file)

print(files)
dsDaily = xr.open_mfdataset(files)

['/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/observations/RS_Soil_moisture/ESA_SM/3_daily_correlation/ESA_SM_dailyCorrelation_year_1995.nc', '/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/observations/RS_Soil_moisture/ESA_SM/3_daily_correlation/ESA_SM_dailyCorrelation_year_2020.nc', '/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/observations/RS_Soil_moisture/ESA_SM/3_daily_correlation/ESA_SM_dailyCorrelation_year_2014.nc', '/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/observations/RS_Soil_moisture/ESA_SM/3_daily_correlation/ESA_SM_dailyCorrelation_year_2004.nc', '/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/observations/RS_Soil_moisture/ESA_SM/3_daily_correlation/ESA_SM_dailyCorrelation_year_2010.nc', '/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/observations/RS_Soil_moisture/ESA_SM/3_daily_correlation/ESA_SM_dailyCorrelation_year_2000.nc', '/Users/darrieythorsson/compHydro/data/

In [181]:
shp = gpd.read_file(layer_polygon)
correlation_map = xr.corr(dsDaily.ESA_SM, dsDaily.scalarTotalSoilWat, dim ='time')
shp['ESA_SM_correlation'] = abs(correlation_map) 



In [182]:
save_corr_file_path = read_from_control(controlFolder/controlFile,'observation_soil_correlation_path')
save_corr_file_name = read_from_control(controlFolder/controlFile,'observation_soil_correlation_name')

# Specify default path if needed
if save_corr_file_path == 'default':
    save_corr_file_path = make_default_path('evaluation/ESA_SM') # outputs a Path()
else:
    save_corr_file_path = Path(save_corr_file_path) # make sure a user-specified path is a Path()


shp.to_file(save_corr_file_path/Path(save_corr_file_name))

/var/folders/pc/q8630ffx72z0bwfzz59ws0kc0000gn/T/ipykernel_17723/2974645748.py:11: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp.to_file(save_corr_file_path/Path(save_corr_file_name))


#### Code provenance
Generates a basic log file in the domain folder and copies the control file and itself there.

In [183]:
# Set the log path and file name
logPath = intersect_path
log_suffix = '_catchment_dem_intersect_log.txt'

In [184]:
# Create a log folder
logFolder = '_workflow_log'
Path( logPath / logFolder ).mkdir(parents=True, exist_ok=True)

In [185]:
# Copy this script
thisFile = '1_find_HRU_elevation.ipynb'
copyfile(thisFile, logPath / logFolder / thisFile);

FileNotFoundError: [Errno 2] No such file or directory: '1_find_HRU_elevation.ipynb'

In [ ]:
# Get current date and time
now = datetime.now()

In [ ]:
# Create a log file 
logFile = now.strftime('%Y%m%d') + log_suffix
with open( logPath / logFolder / logFile, 'w') as file:
    
    lines = ['Log generated by ' + thisFile + ' on ' + now.strftime('%Y/%m/%d %H:%M:%S') + '\n',
             'Found mean HRU elevation from MERIT Hydro adjusted elevation DEM.']
    for txt in lines:
        file.write(txt)  